In [ ]:
import nfl_data_py as nfl
import pandas as pd
import os
import urllib.request
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox
from matplotlib.offsetbox import OffsetImage
from PIL import Image
import numpy as np
from io import BytesIO
import requests
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# Get Data
nfl_data = nfl.import_schedules([2023])


In [ ]:
# upload csv cleaned data
test_data = pd.read_csv('nfl_model_data_1011.csv')

In [30]:
# Remove duplicates based on all columns
test_data_df = test_data.drop_duplicates()
test_data_df.head()

,team,defense_passing_epa_x,defense_rushing_epa_x,defense_pacr_x,defesne_wopr_x,defensive_racr_x,defense_sacks_x,defense_interceptions_x,passing_epa_x,rushing_epa_x,...,defensive_racr_y,defense_sacks_y,defense_interceptions_y,passing_epa_y,rushing_epa_y,pacr_y,wopr_y,racr_y,sacks_y,interceptions_y
0,CIN,-2.044480,0.041143,0.736746,0.282051,0.821360,3.750000,1.500000,-4.000764,-1.016093,...,1.12308,3.0,1.0,-1.800275,0.11427,0.84081,0.282051,0.788139,2.25,2.0
1,DAL,-4.439722,-0.891837,0.935361,0.255814,0.888502,2.500000,2.333333,-0.013147,-0.400317,...,1.12308,3.0,1.0,-1.800275,0.11427,0.84081,0.282051,0.788139,2.25,2.0
2,NYG,2.328315,0.389381,1.417280,0.289474,0.655171,1.666667,2.000000,-6.288094,-0.433977,...,1.12308,3.0,1.0,-1.800275,0.11427,0.84081,0.282051,0.788139,2.25,2.0
3,SF,-6.251096,-0.186934,1.926877,0.261905,2.850395,2.166667,2.000000,11.096799,0.009896,...,1.12308,3.0,1.0,-1.800275,0.11427,0.84081,0.282051,0.788139,2.25,2.0
4,WAS,4.278720,0.495720,0.862084,0.305555,1.068466,4.000000,1.000000,-4.242919,0.113383,...,1.12308,3.0,1.0,-1.800275,0.11427,0.84081,0.282051,0.788139,2.25,2.0


In [ ]:
# create stats to show upcpoming games
grouped = nfl_data.groupby('week')

# Select the group corresponding to week 6
week_6_data = grouped.get_group(6)

In [ ]:
week_6_data.head(30)

In [29]:
# Create a new DataFrame with the selected columns
week_df = week_6_data[['home_team', 'home_score', 'away_team', 'away_score']]
week_df.head()
team_matchups = week_df.rename(columns={'home_team': 'opponent_x', 'home_score': 'team_score_x', 'away_team': 'opponent_y', 'away_score': 'opponent_score_x'})
team_matchups.head()

,opponent_x,team_score_x,opponent_y,opponent_score_x
6499,KC,NaN,DEN,NaN
6500,TEN,NaN,BAL,NaN
6501,ATL,NaN,WAS,NaN
6502,CHI,NaN,MIN,NaN
6503,CIN,NaN,SEA,NaN


In [46]:
df_encoded = pd.get_dummies(team_matchups, columns=['opponent_x'], prefix='opponent_x')
df_encoded = pd.get_dummies(team_matchups, columns=['opponent_y'], prefix='opponent_y')
df_encoded.head()

,opponent_x,team_score_x,opponent_score_x,opponent_y_ARI,opponent_y_BAL,opponent_y_CAR,opponent_y_DAL,opponent_y_DEN,opponent_y_DET,opponent_y_IND,opponent_y_MIN,opponent_y_NE,opponent_y_NO,opponent_y_NYG,opponent_y_PHI,opponent_y_SEA,opponent_y_SF,opponent_y_WAS
6499,KC,NaN,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
6500,TEN,NaN,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6501,ATL,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6502,CHI,NaN,NaN,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6503,CIN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [31]:
column_names_test = [
    'defense_passing_epa_x', 'defense_rushing_epa_x', 'defense_pacr_x',
    'defesne_wopr_x','defensive_racr_x','defense_sacks_x','defense_interceptions_x', 'passing_epa_x', 'rushing_epa_x', 'pacr_x','wopr_x',
     'racr_x','sacks_x','interceptions_x','defense_passing_epa_y', 'defense_rushing_epa_y', 'defense_pacr_y',
    'defesne_wopr_y','defensive_racr_y','defense_sacks_y','defense_interceptions_y', 'passing_epa_y', 'rushing_epa_y', 'pacr_y','wopr_y',
     'racr_y','sacks_y','interceptions_y'
]

In [47]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# model
# Split data into features (X) and target variable (y)
X = test_data_df[column_names_test]
y = test_data_df[['team_score_x', 'opponent_score_x']]

# Encode categorical variables using one-hot encoding if needed

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Create a Linear Regression model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(df_encoded)
# Evaluate the model
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")

/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- opponent_score_x
- opponent_x
- opponent_y_ARI
- opponent_y_BAL
- opponent_y_CAR
- ...
Feature names seen at fit time, yet now missing:
- defense_interceptions_x
- defense_interceptions_y
- defense_pacr_x
- defense_pacr_y
- defense_passing_epa_x
- ...

  warnings.warn(message, FutureWarning)


ValueError: could not convert string to float: 'KC'

In [34]:
# Specify the column names in a list
column_names = [
    'defense_passing_epa_x', 'defense_rushing_epa_x', 'defense_pacr_x',
    'defesne_wopr_x', 'defensive_racr_x', 'defense_sacks_x', 'defense_interceptions_x',
    'passing_epa_x', 'rushing_epa_x', 'pacr_x', 'wopr_x', 'racr_x', 'sacks_x', 'interceptions_x',
    'defense_passing_epa_y', 'defense_rushing_epa_y', 'defense_pacr_y',
    'defesne_wopr_y', 'defensive_racr_y', 'defense_sacks_y', 'defense_interceptions_y',
    'passing_epa_y', 'rushing_epa_y', 'pacr_y', 'wopr_y', 'racr_y', 'sacks_y', 'interceptions_y'
]

# Values corresponding to each column
values = [
   4.547433, -0.490836, 0.991264, 0.275000, 1.896770, 5.333333, 1.000000, 
    5.963441, 0.010871, 1.148992, 0.258824, 0.058333, 2.250000, 1.000000,
    
    -4.439722, -0.891837, 0.935361, 0.255814, 0.888502, 2.500000, 
    2.333333, -0.013147, -0.400317, 2.056749, 0.244444, 1.791781, 2.000000, 2.000000,
    
  


    
    ]

# Create a dictionary with the specified column names and values
average_values_model = {column: value for column, value in zip(column_names, values)}


# Create a sample untrained dataset for X using the average values
data_untrained_test = {feature: [average_values_model[feature] for _ in range(4)] for feature in average_values_model}
X_untrained = pd.DataFrame(data_untrained_test)

In [35]:
# 2. Use the trained model to make predictions on X_untrained.
predictions_untrained_team = model.predict(X_untrained)
print(predictions_untrained_team)

[[31.81991218 19.68240727]
 [31.81991218 19.68240727]
 [31.81991218 19.68240727]
 [31.81991218 19.68240727]]


In [36]:
import pandas as pd

# Replace NaN values in the 'team_score_x' and 'opponent_score_x' columns with 0
test_data_df['team_score_x'].fillna(0, inplace=True)
test_data_df['opponent_score_x'].fillna(0, inplace=True)

# Use your trained Linear Regression model
model = LinearRegression()

# Features for prediction (X)
X = test_data_df[column_names_test]

# Make predictions
predictions = model.predict(X)

# Create a new DataFrame with predictions
prediction_df = pd.DataFrame(predictions, columns=['predicted_team_score_x', 'predicted_opponent_score_x'])

# Concatenate the prediction_df with your original test_data_df
final_df = pd.concat([test_data_df, prediction_df], axis=1)

# Now, 'final_df' contains the predictions for 'team_score_x' and 'opponent_score_x'


/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.